### Instructions

1. Lookup the lat-long of the place of interest on Bing maps.
2. Find the quadkey of the building footprint tile using the function coords_to_quadkey().
3. Download the corresponding building footprint tile in GeoJSON format using the function download_bf().
4. Use the 'JSON to Features' tool in ArcGIS Pro to create a feature layer with building footprint polygons.

### Convert Latitude-Longitude to Microsoft Bing Maps Quadkey


In [1]:
# Based on calculations and C# code provided by Microsoft:
# https://learn.microsoft.com/en-us/bingmaps/articles/bing-maps-tile-system#tile-coordinates-and-quadkeys

import math

def coords_to_pixel(latitude: float, longitude: float, detail: int):
    
    latitude = max(-85.05112878, min(85.05112878, latitude))
    longitude = max(-180, min(180, longitude))
    sinLatitude = math.sin(latitude * math.pi / 180)
    
    pixelX = ((longitude + 180) / 360) * 256 * (2 ** detail)
    pixelY = (0.5 - math.log((1 + sinLatitude) / (1 - sinLatitude)) / (4 * math.pi)) * 256 * (2 ** detail)
    
    return int(pixelX), int(pixelY)

def pixel_to_tile(pixelX, pixelY):
    x = pixelX // 256
    y = pixelY // 256
    return x, y

def tile_to_quadkey(x: int, y: int, zoom: int):
    quadkey = ''
    for i in range(zoom, 0, -1):
        digit = 0
        mask = 1 << (i - 1)
        if (x & mask) != 0:
            digit += 1
        if (y & mask) != 0:
            digit += 2
        quadkey += str(digit)
    return quadkey

In [2]:
# Main method to convert lat-longs to MS Quadkeys
def coords_to_quadkey(lat: float, long: float):
    pixelX, pixelY = coords_to_pixel(lat, long, 9)
    x, y = pixel_to_tile(pixelX, pixelY)
    quadkey = int(tile_to_quadkey(x, y, 9))
    return quadkey

In [3]:
lat = 37.574296 
long = 36.923531
# Quadkey: 122110320

In [4]:
quadkey = coords_to_quadkey(lat, long)
quadkey

122110320

## Download building footprints

In [5]:
# Source: https://github.com/microsoft/GlobalMLBuildingFootprints/blob/main/scripts/make-gis-friendly.py

import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
# Install shapely=1.8.4 using 'pip install shapely==1.8.4', if you encounter an error with shapely

def download_bf(quadkey: int):
    dataset_links = pd.read_csv("https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv")
    links = dataset_links[dataset_links.QuadKey == quadkey]
    for _, row in links.iterrows():
        df = pd.read_json(row.Url, lines=True)
        df['geometry'] = df['geometry'].apply(shape)
        gdf = gpd.GeoDataFrame(df, crs=4326)
        gdf.to_file(f"{row.QuadKey}.geojson", driver="GeoJSON")

In [6]:
# this is the quadkey of the geography you want to retrieve
# quadkey = 122110320

In [7]:
# The file downloads in the location of the notebook
download_bf(quadkey)